In [1]:
%load_ext sql

In [2]:
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
import peakutils
import locate
import farwave
import time

from scipy import signal

MIC_IND_LR = {
    0: (1, 2),
    1: (2, 0),
    2: (0, 1)
}

ANGLE_OFFSET = {
    0: 0,
    1: 2.0944, # 120 deg in rad
    2: 4.18879, # 240 deg in rad
}


In [3]:
%matplotlib notebook

In [4]:
%sql sqlite:///schema/sunday_cardboard.db

u'Connected: None@schema/sunday_cardboard.db'

In [5]:
l = 0.3 # Length from mic to centre of array

def calc_delays(sigs, offsets):
    """ Calculate matrix of delays given N sigs
    """
    results = []
    for j in range(len(sigs)):
        for k in range(len(sigs)):
            if j != k:
                results.append((
                        (j, k), locate.xcorr_peaks(sigs[j], sigs[k], offsets[j], offsets[k], l)
                    ))
            

    # 3x3 array delays[i][j] is the delay of signal j relative to signal i
    delays = np.zeros((len(sigs), len(sigs)), dtype=np.float)
    for res in results:
        key, val = res[0], res[1]
        delays[key[0]][key[1]] = val[1] # xcorr (val, delay) tuple
        delays[key[1]][key[0]] = -delays[key[0]][key[1]]
        
    return delays

def calc_angles(sigs, delays, l):
    """ Given delays matrix, calculate angle using both farwave and solver
    """
    # Estimate "location" of sound source, create array record
    theta = 0             
    for j in range(len(sigs)):
        if delays[j][(j+1)%3] >= 0 and delays[j][(j+2)%3] >= 0:
            lr = MIC_IND_LR[j]
            near_pair = (min(j, lr[0]), max(j, lr[0])) if delays[j][lr[0]] < delays[j][lr[1]] else (min(j, lr[1]), max(j, lr[1]))
            print 'Near Pair: %r' %  str(near_pair)
            f_theta = np.deg2rad(farwave.calc_angle(delays, l, near_pair=near_pair))
            print 'Using microphone %d as closest mic - (%d left, %d right)' % (j, lr[0], lr[1])
            r, theta = 0, 0 #locate.locate(delays[j][lr[0]], delays[j][lr[1]], l, r0=5., theta0=f_theta-ANGLE_OFFSET[j])
            theta += ANGLE_OFFSET[j]
            print 'Solver: r = %f, theta = %f' % (r, np.rad2deg(theta))
            break
    
    return np.rad2deg(theta), np.rad2deg(f_theta)


In [6]:
experiments = %sql select id, comment from experiment

Done.


In [7]:
ang_solver_raw = []
ang_solver_butter = []
ang_far_raw = []
ang_far_butter = []

for i in range(len(experiments)):
    cur_experiment_id = int(experiments[i][0])
    result = %sql select data from mic where experiment_id = :cur_experiment_id and array_id = 0
        
    mic_0 = np.array(result[0][0])
    mic_1 = np.array(result[1][0])
    mic_2 = np.array(result[2][0])
    
    bufs = [mic_0, mic_1, mic_2]
    #try:
    try: 
        sigs, sigs_filt = [], []
        for buf_i in bufs:
            sig, sig_filt = locate.preprocess_sig(buf_i)
            sigs.append(sig)
            sigs_filt.append(sig_filt)
                
        sigs_cropped, sigs_butter_cropped, offsets, _, _ = locate.crop_sigs_npeaks(sigs, sigs_filt)
    except RuntimeError as e:
        print 'Error: %s' % str(e)
        solver_raw = solver_butter = far_raw = far_butter = 0
        ang_solver_raw.append(solver_raw)
        ang_solver_butter.append(solver_butter)
        ang_far_raw.append(far_raw)
        ang_far_butter.append(far_butter)
        continue
        
    # Calculate delays, angle using raw signal (cropped)
    delays_raw = calc_delays(sigs_cropped, offsets)
    solver_raw, far_raw = calc_angles(sigs_cropped, delays_raw, l)

    delays_butter = calc_delays(sigs_butter_cropped, offsets)
    solver_butter, far_butter = calc_angles(sigs_butter_cropped, delays_butter, l)
    #except:
    #    print 'Failure to find peaks: Setting to 0'
    #    solver_raw = far_raw = solver_butter = far_butter = 0
        
    print 'ID: %d, %f,%f,%f,%f' % (cur_experiment_id, far_raw, far_butter, solver_raw, solver_butter)
    
    ang_solver_raw.append(solver_raw)
    ang_solver_butter.append(solver_butter)
    ang_far_raw.append(far_raw)
    ang_far_butter.append(far_butter)


Done.
Near Pair: '(0, 2)'
Angles: [11.707039944049448, 347.335066534627, 359.34489092913606]
Errors: [1.5030314132153986, 1.4755579237395815, 1.0000653695691588]

DEL_ERR: 0.027473
Error: 0.027473, 1.000065, 'False'
DEL_ERR: 0.502966
Error: 0.502966, 1.000065, 'True'
DEL_ERR: 0.475493
Error: 0.475493, 1.000065, 'True'
Angle (CCW): 359.355950
Using microphone 0 as closest mic - (1 left, 2 right)
Solver: r = 0.000000, theta = 0.000000
Near Pair: '(0, 2)'
Angles: [10.899881992708337, 348.32262614676409, 359.49091360987421]
Errors: [1.527325556529274, 1.5039053682124877, 1.0000394749873351]

DEL_ERR: 0.023420
Error: 0.023420, 1.000039, 'False'
DEL_ERR: 0.527286
Error: 0.527286, 1.000039, 'True'
DEL_ERR: 0.503866
Error: 0.503866, 1.000039, 'True'
Angle (CCW): 359.489288
Using microphone 0 as closest mic - (1 left, 2 right)
Solver: r = 0.000000, theta = 0.000000
ID: 1, 359.355950,359.489288,0.000000,0.000000
Done.
Near Pair: '(0, 2)'
Angles: [21.641451949378286, 338.02213686572725, 359.73164

locate.py:360: RuntimeWarning: invalid value encountered in divide
  CORR = CORR / (np.abs(SIG1) * np.abs(np.conj(SIG2)))


Near Pair: '(1, 2)'
Angles: [251.44944104353004, 210.54680969489701, 269.45319030510302]
Errors: [1.0203037909921615, 104.78352855705414, 104.78352855705414]

DEL_ERR: 103.763225
Error: 103.763225, 1.020304, 'True'
DEL_ERR: 103.763225
Error: 103.763225, 1.020304, 'True'
DEL_ERR: 0.000000
Error: 0.000000, 1.020304, 'False'
Angle (CCW): 246.121260
Using microphone 2 as closest mic - (0 left, 1 right)
Solver: r = 0.000000, theta = 239.999988
Near Pair: '(0, 2)'
Angles: [252.32457484542391, 278.23724394906043, 226.04631479455676]
Errors: [1.023589528312158, 1.076742645441642, 1.4407630245727059]

DEL_ERR: 0.053153
Error: 0.053153, 1.023590, 'True'
Setting final_ang to 278.237244
DEL_ERR: 0.364020
Error: 0.364020, 1.023590, 'False'
Angle (CCW): 278.237244
Using microphone 2 as closest mic - (0 left, 1 right)
Solver: r = 0.000000, theta = 239.999988
ID: 14, 246.121260,278.237244,239.999988,239.999988
Done.
Near Pair: '(1, 2)'
Angles: [237.85268066908992, 242.07827993192518, 234.0615207722586

In [8]:
# Assuming comment is the _actual_ angle
for i in range(len(ang_solver_raw)):
    print '%f,%f,%f,%f,%s' % (ang_far_raw[i], ang_far_butter[i], ang_solver_raw[i], ang_solver_butter[i], experiments[i][1])

359.355950,359.489288,0.000000,0.000000,0
359.752469,359.870150,0.000000,0.000000,0
5.422860,4.666675,0.000000,0.000000,0
324.552961,323.731278,0.000000,0.000000,30
325.681829,324.804857,0.000000,0.000000,30
325.642425,324.678846,0.000000,0.000000,30
294.851317,294.312880,239.999988,239.999988,60
294.815745,294.676160,239.999988,239.999988,60
295.202085,295.222764,239.999988,239.999988,60
0.000000,0.000000,0.000000,0.000000,90
267.447911,267.924202,239.999988,239.999988,90
268.878670,269.398165,239.999988,239.999988,90
267.896254,267.928859,239.999988,239.999988,90
246.121260,278.237244,239.999988,239.999988,120
237.802802,239.155101,239.999988,239.999988,120
240.541113,197.401357,239.999988,239.999988,120
212.655150,211.671502,239.999988,239.999988,150
211.015363,211.959439,239.999988,239.999988,150
212.669213,212.528683,239.999988,239.999988,150
211.833937,212.094429,239.999988,239.999988,150
211.574178,211.588075,239.999988,239.999988,150
212.038547,211.305903,239.999988,239.999988,